In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import random
import time
import datetime
import numpy as np

from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler
from data.token_classification import TokenClassificationDataset
from transformers import LayoutLMv3Tokenizer

from src.modeling.docpolarbert.modeling_docpolarbert import DocPolarBERTForTokenClassification, DocPolarBERTModel
from src.modeling.docpolarbert.train_utils import train_step_token_classification, eval_token_classification

In [ ]:
# Load PAYSLIPS dataset
DATA_DIR = '../data/payslips'
pad_token_label_id = CrossEntropyLoss().ignore_index
SEED = 6
NUM_EPOCHS = 30
LEARNING_RATE = 5e-5
BATCH_SIZE = 16
# Set all seeds
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

g = torch.Generator()
g.manual_seed(SEED)

tokenizer = LayoutLMv3Tokenizer.from_pretrained('microsoft/layoutlmv3-base')
train_dataset = TokenClassificationDataset(
    data_dir=DATA_DIR,
    tokenizer=tokenizer,
    pad_token_label_id=pad_token_label_id,
    mode='train')

val_dataset = TokenClassificationDataset(
    data_dir=DATA_DIR,
    tokenizer=tokenizer,
    pad_token_label_id=pad_token_label_id,
    mode='test')

train_sampler = RandomSampler(train_dataset)

train_sampler = torch.utils.data.RandomSampler(train_dataset, generator=g)

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              sampler=train_sampler)

val_sampler = torch.utils.data.RandomSampler(val_dataset, generator=g)

val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=BATCH_SIZE,
                            sampler=val_sampler)



In [ ]:
DOCPOLARBERT_DIR = '../models/docpolarbert-base'

model = DocPolarBERTForTokenClassification.from_pretrained(DOCPOLARBERT_DIR, num_labels=len(train_dataset.idx2label))
model.to("cpu" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)


with torch.no_grad():
    avg_eval_loss, precision, recall, f1 = eval_token_classification(
    model=model,
    eval_dataloader=val_dataloader,
    idx2label=val_dataset.idx2label,
    print_results=True)
    print(f"Epoch [{0}/{NUM_EPOCHS}], Average validation loss: {avg_eval_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}.")


In [ ]:
for epoch in range(NUM_EPOCHS):
    start_epoch = time.time()
    print(f'Starting epoch {epoch + 1}/{NUM_EPOCHS}')
    avg_loss  = train_step_token_classification(
            model=model,
            train_dataloader=train_dataloader,
            idx2label=train_dataset.idx2label,
            optimizer=optimizer,
            )
    # ---------------------------------------- Validation ---------------------------------------- #
    model.eval()
    with torch.no_grad():
        avg_eval_loss, precision, recall, f1 = eval_token_classification(
            model=model,
            eval_dataloader=val_dataloader,
            idx2label=val_dataset.idx2label,
            print_results=True)
        print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}], Average training loss: {avg_loss:.4f}, Average validation loss: {avg_eval_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}.")
        print(f"Time taken for epoch {epoch + 1}: {datetime.timedelta(seconds=int(time.time() - start_epoch))}")
    model.train()

In [ ]:
# Run evaluation on the validation set
model.eval()
with torch.no_grad():
    avg_eval_loss, precision, recall, f1 = eval_token_classification(
        model=model,
        eval_dataloader=val_dataloader,
        idx2label=val_dataset.idx2label,
        print_results=True)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}.")

In [ ]:
model.config.id2label = val_dataset.idx2label
model.config.label2id = val_dataset.label2idx
# Save fine-tuned model
model.save_pretrained('models/docpolarbert-payslips')